In [2]:
import pandas as pd 
import numpy as np

import tensorflow as tf 
from tensorflow.keras import *
from tensorflow.keras.layers import *

df = pd.read_csv(
    './aws_a5_1_all.csv'
)

df.head()

,num,cputime_dif,depth,high,image,tension,time,velocity,volume,weld_velocity,width,wire_l
0,0,0.109213,-2.612515,2.251549,4482.0,20.0,2.614,5.5,0.704238,8.0,4.850269,15.0
1,1,0.113643,-2.568461,2.261362,4497.0,20.0,2.629,5.5,0.093257,8.0,5.014591,15.0
2,2,0.187733,-2.284407,2.307767,5361.0,20.0,3.493,5.5,5.371661,8.0,6.496406,15.0
3,3,0.124918,-2.072830,2.291701,5433.0,20.0,3.565,5.5,0.447639,8.0,7.534782,15.0
4,4,0.110063,-2.497762,2.195335,6325.0,20.0,4.457,5.5,5.545741,8.0,7.738949,15.0


In [3]:
features = np.array(df[['depth' , 'high' , 'tension' , 'velocity' , 'volume' , 'weld_velocity' , 'width' , 'wire_l']])
labels = np.array(df.pop('depth'))

features

array([[-2.61251488,  2.25154901, 20.        , ...,  8.        ,
         4.85026926, 15.        ],
       [-2.56846062,  2.26136213, 20.        , ...,  8.        ,
         5.01459088, 15.        ],
       [-2.28440721,  2.30776663, 20.        , ...,  8.        ,
         6.49640607, 15.        ],
       ...,
       [-3.09060336,  0.05518935, 29.        , ..., 12.        ,
         5.40717952, 13.6       ],
       [-3.09060336,  0.05518935, 29.        , ..., 12.        ,
         5.40717952, 15.        ],
       [-3.09058904,  0.05518909, 29.        , ..., 12.        ,
         5.40714196, 15.        ]])

In [4]:
numeric_dataset = tf.data.Dataset.from_tensor_slices((features, labels))

for row in numeric_dataset.take(3):
    print(row)

(<tf.Tensor: shape=(8,), dtype=float64, numpy=
array([-2.61251488,  2.25154901, 20.        ,  5.5       ,  0.7042383 ,
        8.        ,  4.85026926, 15.        ])>, <tf.Tensor: shape=(), dtype=float64, numpy=-2.612514882967101>)
(<tf.Tensor: shape=(8,), dtype=float64, numpy=
array([-2.56846062,  2.26136213, 20.        ,  5.5       ,  0.09325717,
        8.        ,  5.01459088, 15.        ])>, <tf.Tensor: shape=(), dtype=float64, numpy=-2.5684606173793463>)
(<tf.Tensor: shape=(8,), dtype=float64, numpy=
array([-2.28440721,  2.30776663, 20.        ,  5.5       ,  5.37166119,
        8.        ,  6.49640607, 15.        ])>, <tf.Tensor: shape=(), dtype=float64, numpy=-2.284407206321464>)


In [5]:
mean = np.mean(features , axis=0)
mean

array([-2.57594384,  1.17014343, 28.27534791,  6.60735586,  0.83287074,
        9.5387674 ,  6.60738607, 13.73389776])

In [6]:
standard = np.std(features , axis= 0)

In [7]:
features

array([[-2.61251488,  2.25154901, 20.        , ...,  8.        ,
         4.85026926, 15.        ],
       [-2.56846062,  2.26136213, 20.        , ...,  8.        ,
         5.01459088, 15.        ],
       [-2.28440721,  2.30776663, 20.        , ...,  8.        ,
         6.49640607, 15.        ],
       ...,
       [-3.09060336,  0.05518935, 29.        , ..., 12.        ,
         5.40717952, 13.6       ],
       [-3.09060336,  0.05518935, 29.        , ..., 12.        ,
         5.40717952, 15.        ],
       [-3.09058904,  0.05518909, 29.        , ..., 12.        ,
         5.40714196, 15.        ]])

In [8]:
def normalize(data , label) :
    return (data - mean) / standard , label

x_test = numeric_dataset.take(10)

numeric_batches = numeric_dataset.shuffle(1000).batch(4)
numeric_batches = numeric_batches.map(normalize)

In [9]:
def get_basic_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer= optimizers.Adam() ,
                loss= losses.mean_squared_error ,
                metrics= [metrics.MeanSquaredError()])
  return model

model = get_basic_model()

model.fit(numeric_batches , batch_size=4 , epochs=10)

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

252/252 [==============================] - 1s 3ms/step - loss: 3.3678 - mean_squared_error: 3.3678
Epoch 2/10
252/252 [==============================] - 1s 3ms/step - loss: 0.2459 - mean_squared_error: 0.2459
Epoch 3/10
252/252 [==============================] - 1s 3ms/step - loss: 0.0303 - mean_squared_error: 0.0303
Epoch 4/10
252/252 [==============================] - 1s 3ms/step - loss: 0.0163 - mean_squared_error: 0.0163
Epoch 5/10
252/252 [==============================] - 0s 2ms/step - 

In [10]:
test = (np.array([[-2.61251488,  2.25154901, 20.,5.5,0.7042383 ,8.,4.85026926, 15.]]) - mean) / standard
model.predict(test)

array([[-2.3642116]], dtype=float32)

In [14]:
x_test = x_test.map(normalize)

In [15]:
x , y = [] , []

for u , v in x_test.as_numpy_iterator() :
    x.append(u)
    y.append(v)

x = np.array(x)
y = np.array(y)

In [16]:
model.evaluate(x , y)

1/1 [==============================] - 0s 2ms/step - loss: 0.0190 - mean_squared_error: 0.0190


[0.018959669396281242, 0.018959669396281242]